In [1]:
import pandas as pd
import numpy as np
import pickle

ED BUCKETS

In [2]:
vital_df = pd.read_csv('data/mimiciv/3.1/ed/vitalsign.csv')
triage_df = pd.read_csv('data/mimiciv/3.1/ed/triage.csv')
med_df = pd.read_csv('data/mimiciv/3.1/ed/pyxis.csv')

In [3]:
NUM_COLS = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

In [4]:
num_df = pd.concat([vital_df[NUM_COLS], triage_df[NUM_COLS]])

In [5]:
scale = 10_000

In [6]:
buckets = {}
for c in num_df.columns:
    df_c = pd.to_numeric(num_df[c], 'coerce').dropna()
    bucket = np.percentile(df_c, np.arange(0, 110, 10), method='lower')
    bucket = bucket[np.insert(np.diff(bucket) != 0, 0, True)]
    bucket = np.array([
            np.floor(bucket[0] * scale) / scale,             # floor first
            *[round(b, ndigits=4) for b in bucket[1:-1]],         # round middle
            np.ceil(bucket[-1] * scale) / scale              # ceil last
            ])
    buckets[c] = bucket[np.concatenate(([True], bucket[1:] != bucket[:-1]))]

In [7]:
buckets['acuity'] = [1, 2, 3, 4, 5]

In [9]:
with open(f'ed_buckets.pkl', 'wb') as f:
    pickle.dump(buckets, f)

ICU

In [ ]:
COLS = {
    'chartevents': ('subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'valuenum'),
    'inputevents': ('subject_id', 'hadm_id', 'stay_id', 'starttime', 'itemid', 'amount'),
    'outputevents': ('subject_id', 'hadm_id', 'stay_id', 'charttime', 'itemid', 'value'),
    'procedureevents': ('subject_id', 'hadm_id', 'stay_id', 'starttime', 'itemid', 'value'),
    'labevents': ('subject_id', 'hadm_id', 'charttime', 'itemid', 'valuenum'),
}


In [18]:
FNAME = 'procedureevents'
VAL_COL = 'value'
scale = 10_000

In [19]:
codes = []
df_final = None
for i in range(0, 7):
    df = pd.read_parquet(f'data/raw_data/{FNAME}_{i}.parquet', columns=['itemid', VAL_COL])
    codes += list(df['itemid'])
    codes = list(set(codes))
    if df_final is None: df_final = df
    else: df_final = pd.concat([df_final, df])
print(len(codes))

159


In [20]:
buckets = {}
for i, c in enumerate(codes):
    if i % 50 == 0:
        print(i)
    df_c = df_final[df_final['itemid']==c]
    df_values = pd.to_numeric(df_c[VAL_COL], 'coerce').dropna()
    if len(df_values) > 0:
        bucket = np.percentile(df_values, np.arange(0, 110, 10), method='lower')
        bucket = bucket[np.insert(np.diff(bucket) != 0, 0, True)]
        bucket = np.array([
                np.floor(bucket[0] * scale) / scale,             # floor first
                *[round(b, ndigits=4) for b in bucket[1:-1]],         # round middle
                np.ceil(bucket[-1] * scale) / scale              # ceil last
                ])
        buckets[c] = bucket[np.concatenate(([True], bucket[1:] != bucket[:-1]))]

0
50
100
150


In [21]:
with open(f'{FNAME}_buckets.pkl', 'wb') as f:
    pickle.dump(buckets, f)